In [1]:
import numpy as np
import random
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D,Conv1D, MaxPooling1D, MaxPooling2D


In [2]:
def process():
    ###Reading dataset###
    datadir="rice"
    categories=["Bacterial leaf blight","Brown spot","Leaf smut"]
    for category in categories:
        path=os.path.join(datadir,category)
        for img in os.listdir(path):
            img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
    
    
    
    ###Preparing Training Data
    training_data=[]
    def create_training_data():
        for category in categories:
            path=os.path.join(datadir,category)
            class_num=categories.index(category)
            for img in os.listdir(path):
              try:
                gray_img=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                resized_img=cv2.resize(gray_img,(100,100))
                training_data.append([resized_img,class_num])
              except Exception as e:
                pass
    create_training_data()  
    random.seed(2)
    random.shuffle(training_data)
    
    ###converting the format of the input to make that of CNN
    X = [] # feature set
    y = [] # label
    for img, label in training_data:
        temp = np.zeros(3)
        temp[label] = 1
        y.append(temp)
        X.append(img)   
    y = np.array(y)
    X = np.array(X)/255.0
    X = X[:, :, :, np.newaxis]
    
    model = Sequential()
    model.add(Conv2D(24,(2,2), input_shape=(100,100,1), activation='relu'))
    # model.add(activation='relu')
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(24,(4,4),activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation('sigmoid'))
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
    # history = model.fit(X,y,batch_size=40,epochs=40,validation_data=(X,y))
    # history = model.fit(X,y, validation_data=(X[0],y[0]), epochs=50)
    x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
    history = model.fit(x_train,y_train, epochs=50,batch_size=40)
    res=model.evaluate(x_test, y_test)
    print(res)

    
    
####hu moments model
def model2():
    data=[]

    datadir="rice"
    categories=["Bacterial leaf blight","Brown spot","Leaf smut"]
    image_list = []
    for category in categories:
      path=os.path.join(datadir,category)
      class_num = categories.index(category)
      for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array,(100,100))
        data.append([new_array,class_num])
        image_list.append(new_array)
    
    
    random.seed(0)
    random.shuffle(data)
    def fd_hu_moments_image(image):
        feature = cv2.HuMoments(cv2.moments(image)).flatten() 
        new_feature = []
        for f in feature:
          new_feature.append(np.sign(f)*np.log10(abs(f)))
        new_feature = np.array(new_feature)
        return new_feature
    
    
    y_new = []
    x_new = []
    # print(len(data))
    for img,class_num in data:
      temp = np.zeros(3)
      temp[class_num] = 1
      x_new.append(fd_hu_moments_image(img))
      y_new.append(temp)
    y_new = np.array(y_new)
    x_new = np.array(x_new)

    mean = np.mean(x_new, axis=0)
    x_new = (x_new - mean)
    norm = np.linalg.norm(x_new, axis=0)
    x_new = x_new/norm

    model=Sequential()
    model.add(Dense(100,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(50,activation='relu'))

    model.add(Dense(3))
    model.add(Activation('sigmoid'))
    model.compile(loss="categorical_crossentropy",optimizer="adam", metrics=['accuracy'])
    x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,random_state=0,test_size=0.1)
    history = model.fit(x_train,y_train, epochs=330,batch_size=50)
#     print("train accuracy")
#     model.evaluate(x_train,y_train)
    

    
process()
model2()   

Train on 95 samples
Epoch 1/50
95/95 [==============================] - 4s 41ms/sample - loss: 1.1149 - accuracy: 0.3263
Epoch 2/50
95/95 [==============================] - 1s 14ms/sample - loss: 1.0942 - accuracy: 0.3579
Epoch 3/50
95/95 [==============================] - 1s 14ms/sample - loss: 1.1042 - accuracy: 0.3579
Epoch 4/50
95/95 [==============================] - 1s 14ms/sample - loss: 1.0914 - accuracy: 0.4000
Epoch 5/50
95/95 [==============================] - 1s 13ms/sample - loss: 1.0947 - accuracy: 0.3579
Epoch 6/50
95/95 [==============================] - 1s 12ms/sample - loss: 1.0932 - accuracy: 0.4105
Epoch 7/50
95/95 [==============================] - 1s 13ms/sample - loss: 1.0919 - accuracy: 0.3684
Epoch 8/50
95/95 [==============================] - 1s 13ms/sample - loss: 1.0879 - accuracy: 0.3579
Epoch 9/50
95/95 [==============================] - 1s 13ms/sample - loss: 1.0858 - accuracy: 0.3789
Epoch 10/50
95/95 [==============================] - 1s 15ms/sample - l

107/107 [==============================] - 0s 217us/sample - loss: 1.0876 - accuracy: 0.4486
Epoch 23/330
107/107 [==============================] - 0s 167us/sample - loss: 1.0858 - accuracy: 0.4486
Epoch 24/330
107/107 [==============================] - 0s 190us/sample - loss: 1.0854 - accuracy: 0.4486
Epoch 25/330
107/107 [==============================] - 0s 120us/sample - loss: 1.0879 - accuracy: 0.3925
Epoch 26/330
107/107 [==============================] - 0s 179us/sample - loss: 1.0837 - accuracy: 0.4766
Epoch 27/330
107/107 [==============================] - 0s 193us/sample - loss: 1.0775 - accuracy: 0.4766
Epoch 28/330
107/107 [==============================] - 0s 150us/sample - loss: 1.0812 - accuracy: 0.4860
Epoch 29/330
107/107 [==============================] - 0s 230us/sample - loss: 1.0805 - accuracy: 0.4766
Epoch 30/330
107/107 [==============================] - 0s 244us/sample - loss: 1.0749 - accuracy: 0.4673
Epoch 31/330
107/107 [==============================] - 0s 

107/107 [==============================] - 0s 142us/sample - loss: 0.8796 - accuracy: 0.5607
Epoch 100/330
107/107 [==============================] - 0s 204us/sample - loss: 0.8629 - accuracy: 0.5981
Epoch 101/330
107/107 [==============================] - 0s 141us/sample - loss: 0.8499 - accuracy: 0.6262
Epoch 102/330
107/107 [==============================] - 0s 132us/sample - loss: 0.8892 - accuracy: 0.5327
Epoch 103/330
107/107 [==============================] - 0s 135us/sample - loss: 0.8762 - accuracy: 0.5981
Epoch 104/330
107/107 [==============================] - 0s 284us/sample - loss: 0.8597 - accuracy: 0.6355
Epoch 105/330
107/107 [==============================] - 0s 192us/sample - loss: 0.8301 - accuracy: 0.6449
Epoch 106/330
107/107 [==============================] - 0s 169us/sample - loss: 0.8626 - accuracy: 0.5514
Epoch 107/330
107/107 [==============================] - 0s 289us/sample - loss: 0.8713 - accuracy: 0.6542
Epoch 108/330
107/107 [============================

107/107 [==============================] - 0s 151us/sample - loss: 0.7968 - accuracy: 0.5607
Epoch 176/330
107/107 [==============================] - 0s 223us/sample - loss: 0.7445 - accuracy: 0.6729
Epoch 177/330
107/107 [==============================] - 0s 192us/sample - loss: 0.8107 - accuracy: 0.6262
Epoch 178/330
107/107 [==============================] - 0s 239us/sample - loss: 0.7764 - accuracy: 0.6729
Epoch 179/330
107/107 [==============================] - 0s 153us/sample - loss: 0.7444 - accuracy: 0.6729
Epoch 180/330
107/107 [==============================] - 0s 309us/sample - loss: 0.7430 - accuracy: 0.6822
Epoch 181/330
107/107 [==============================] - 0s 175us/sample - loss: 0.8063 - accuracy: 0.5794
Epoch 182/330
107/107 [==============================] - 0s 143us/sample - loss: 0.8386 - accuracy: 0.6449
Epoch 183/330
107/107 [==============================] - 0s 175us/sample - loss: 0.7694 - accuracy: 0.6355
Epoch 184/330
107/107 [============================

107/107 [==============================] - 0s 164us/sample - loss: 0.7212 - accuracy: 0.6916
Epoch 251/330
107/107 [==============================] - 0s 400us/sample - loss: 0.7540 - accuracy: 0.6355
Epoch 252/330
107/107 [==============================] - 0s 158us/sample - loss: 0.7769 - accuracy: 0.6168
Epoch 253/330
107/107 [==============================] - 0s 154us/sample - loss: 0.7716 - accuracy: 0.6449
Epoch 254/330
107/107 [==============================] - 0s 244us/sample - loss: 0.7519 - accuracy: 0.6542
Epoch 255/330
107/107 [==============================] - 0s 191us/sample - loss: 0.7422 - accuracy: 0.6729
Epoch 256/330
107/107 [==============================] - 0s 207us/sample - loss: 0.6889 - accuracy: 0.7103
Epoch 257/330
107/107 [==============================] - 0s 390us/sample - loss: 0.8082 - accuracy: 0.6168
Epoch 258/330
107/107 [==============================] - 0s 157us/sample - loss: 0.7377 - accuracy: 0.6916
Epoch 259/330
107/107 [============================

107/107 [==============================] - 0s 165us/sample - loss: 0.7613 - accuracy: 0.6822
Epoch 326/330
107/107 [==============================] - 0s 212us/sample - loss: 0.7169 - accuracy: 0.6542
Epoch 327/330
107/107 [==============================] - 0s 297us/sample - loss: 0.7261 - accuracy: 0.6449
Epoch 328/330
107/107 [==============================] - 0s 168us/sample - loss: 0.6971 - accuracy: 0.6916
Epoch 329/330
107/107 [==============================] - 0s 178us/sample - loss: 0.6925 - accuracy: 0.6729
Epoch 330/330
107/107 [==============================] - 0s 168us/sample - loss: 0.7018 - accuracy: 0.6542
